In [ ]:
import re

# --- Helper dictionary to convert Z to element symbol ---
Z_TO_ELEMENT = {
    1: "H", 2: "He", 3: "Li", 4: "Be", 5: "B", 6: "C", 7: "N", 8: "O", 9: "F", 10: "Ne",
    11: "Na", 12: "Mg", 13: "Al", 14: "Si", 15: "P", 16: "S", 17: "Cl", 18: "Ar", 19: "K", 20: "Ca",
    21: "Sc", 22: "Ti", 23: "V", 24: "Cr", 25: "Mn", 26: "Fe", 27: "Co", 28: "Ni", 29: "Cu", 30: "Zn",
    31: "Ga", 32: "Ge", 33: "As", 34: "Se", 35: "Br", 36: "Kr", 37: "Rb", 38: "Sr", 39: "Y", 40: "Zr",
    41: "Nb", 42: "Mo", 43: "Tc", 44: "Ru", 45: "Rh", 46: "Pd", 47: "Ag", 48: "Cd", 49: "In", 50: "Sn",
    51: "Sb", 52: "Te", 53: "I", 54: "Xe", 55: "Cs", 56: "Ba", 57: "La", 58: "Ce", 59: "Pr", 60: "Nd",
    61: "Pm", 62: "Sm", 63: "Eu", 64: "Gd", 65: "Tb", 66: "Dy", 67: "Ho", 68: "Er", 69: "Tm", 70: "Yb",
    71: "Lu", 72: "Hf", 73: "Ta", 74: "W", 75: "Re", 76: "Os", 77: "Ir", 78: "Pt", 79: "Au", 80: "Hg",
    81: "Tl", 82: "Pb", 83: "Bi", 84: "Po", 85: "At", 86: "Rn", 87: "Fr", 88: "Ra", 89: "Ac", 90: "Th",
    91: "Pa", 92: "U"
}

def get_pot_dict(feff_file):

    potentials = {}

    with open(feff_file, "r") as f:
        lines = f.readlines()

    pot_section = False

    for line in lines:
        stripped = line.strip()

        # --- Detect start/end of sections ---
        if re.search(r"\bPOTENTIALS\b", stripped):
            pot_section = True
            continue
        if stripped == "" or stripped[0].isalpha():
            pot_section = False
            continue

        # --- Parse POTENTIALS section ---
        if pot_section:
            parts = stripped.split()
            pot_idx, Z = int(parts[0]), int(parts[1])
            potentials[pot_idx] = Z_TO_ELEMENT.get(Z)
    
    return potentials

def feff_to_xyz(feff_file, xyz_file):
    potentials = get_pot_dict(feff_file)
    atoms = []

    with open(feff_file, "r") as f:
        lines = f.readlines()

    pot_section = False

    for line in lines:
        stripped = line.strip()

        # --- Detect start/end of sections ---
        if re.search(r"\bATOMS\b", stripped):
            atom_section = True
            continue
        if stripped == "" or stripped[0].isalpha():
            pot_section = atom_section = False
            continue

        # --- Parse ATOMS section ---
        elif atom_section:
            parts = stripped.split()
            print(parts)
            x, y, z = map(float, parts[0:3])
            pot_idx = int(parts[3])
            elem = potentials.get(pot_idx)
            atoms.append((elem, x, y, z))
        print(atoms)

    # --- Write XYZ ---
    with open(xyz_file, "w") as f:
        f.write(f"{len(atoms)}\n")
        f.write("Generated from feff.inp\n")
        for elem, x, y, z in atoms:
            f.write(f"{elem:2s} {x:.6f} {y:.6f} {z:.6f}\n")

    print(f"✅ Wrote {len(atoms)} atoms to {xyz_file}")

# Example:
feff_to_xyz("feff.inp", "output.xyz")
